# Создание и нормализация базы данных

# 1. Экспортируем из исходных данных 2 .csv файла с каждого листа

Из исходного файла `customer_and_transaction.xlsx` (Excel: Файл -> Сохранить как) получаем 2 файла `customer.csv` и `transaction.csv`

# 2. Загружаем полученные файлы в DBeaver, создав 2 одноименные таблицы

Используем при импорте разделитель `;`

Импорт `transaction.csv`:

<img src="imgs/image01.png" width="500"/>  

<img src="imgs/image02.png" width="900"/>

Импорт `customer.csv`:

<img src="imgs/image03.png" width="900"/>  

Таблица `transaction`:

<img src="imgs/image04.png" width="900"/>  

Таблица `customer`:

<img src="imgs/image05.png" width="900"/>  

# 3. Опишем атрибуты данных в наших таблицах

Таблица `transaction`:
- **transaction_id** - идентификатор транзакции
- **product_id** - идентификатор продукта
- **customer_id** - идентификатор клиента
- **transaction_date** - дата транзакции
- **online_order** - онлайн заказ (значение да/нет)
- **order_status** - статус заказа (подтвержен/отменен)
- **brand** - бренд товара
- **product_line** - продуктовая линейка
- **product_class** - класс продукта
- **product_size** - размер продукта (не совсем понятно - физический размер?)
- **list_price** - прейскурантная цена
- **standard_cost** - стандартная (фактическая) цена

Речь, скорее всего, идет о продажах велосипедов.

Таблица `customer`:
- **customer_id** - идентификатор клиента
- **first_name** - имя
- **last_name** - фамилия
- **gender** - пол
- **DOB** - дата рождения
- **job_title** - название должности
- **job_industry_category** - отраслевая категория
- **wealth_segment** - уровень благосостояния
- **deceased_indicator** - индикатор умершего клиента
- **owns_car** - наличие собственного авто
- **address** - адресс
- **postcode** - почтовый индекс
- **state** - штат
- **country** - страна (все значения Австралия, нужно ли это поле вообще?)
- **property_valuation** - оценка имущества/недвижимости (?)

Можно сразу отметить, что все клиенты проживают в Австралии:

In [ ]:
select distinct c.country 
from customer c 

# 4. Нормализуем БД

## 4.1. 1 нормальная форма

Для того чтобы таблица была в 1НФ все атрибуты в таблице должны быть **простыми**, все сохраняемые данные на пересечении столбцов и строк — содержать лишь **скалярные значения** (не вектора).

Не должно быть составных данных и дубликатов столбцов.

В таблице `transaction`:
- атрибут **transaction_date** имеет тип данных *varchar(50)*, он составной (день, месяц, год) однако, если мы переводем его в тип данных *date*, то это уже не будет составной тип даннх, т.к. PostgreSQL умеет с ними работать как с датами и извлекать необходимую дополнительную информация (отдельно день, например).

В таблице `customer`:
- атрибут **DOB** также имеет тип данных *varchar(50)*, он составной (год, месяц, день) однако, если мы переводем его в тип данных *date*, то это уже не будет составной тип даннх, т.к. PostgreSQL умеет с ними работать как с датами и извлекать необходимую дополнительную информация (отдельно день, например).
- атрибут **address** состоит из улицы и номера дома, в теории если клиентов очень много, то, возможно, имеет смысл разделить этот атрибут отдельно на улицу и номер дома - например, у нас будет несколько клиентов жить на одной улице в одном населенном пункте. Но так как все-же база небольшая, вероятно в данной ситуации это не нужно делать.

Повторяющихся столбцов нет.

## 4.2. 2 нормальная форма

Отношения будут соответствовать 2НФ, если сама БД находится в 1НФ, а **каждый столбец**, который не является ключом, *неприводимо* зависит от **первичного ключа**.

В обеих таблицах есть первичные ключи от которых зависят определнные атрибуты:
- **transaction_id** - идентификатор транзакции
- **product_id** - идентификатор продукта
- **customer_id** - идентификатор клиента, он также является и внешним ключом


## 4.3. 3 нормальная форма

Таблица должна находиться во 2НФ, плюс **любой столбец**, который не является ключом, должен *нетранзитивно* зависеть лишь от **первичного ключа**.

Покажем нетранзитивную зависимость атрибутов от первичных ключей:
- таблица `transaction`:
    - **transaction_id** :
        - *transaction_date*
        - *online_order*
        - *order_status*
    - **product_id** :
        - *brand*
        - *product_line*
        - *product_class*
        - *product_size*
        - *list_price*
        - *standard_cost*

        
- таблица `customer`:
    - **customer_id**
        - *first_name*
        - *last_name*
        - *gender*
        - *DOB*
        - *job_title*
        - *job_industry_category*
        - *wealth_segment*
        - *deceased_indicator*
        - *owns_car*
        - *property_valuation*
    - **addres_id** (создадим, по идее, один и тот же клиент может заказывать по разным адресам)
        - *address*
        - *postcode*
        - *state*
        - *country*

In [ ]:
select 
	count(distinct (c.customer_id )) as count_customer_id,
	count(distinct (c.address )) as count_adress
from customer c 

<img src="imgs/image07.png" width="900"/>  

Видим, что количество клиентов и адресов не совпадают

## 4.4. Итоговое представление данных

<img src="imgs/image0666.png" width="1200"/>  

**Выводы:**
- Разбиваем исходные данные по ключам и таблицам, как представленно выше
- меняем тип данных:
    - *transaction_date* -> timestamp/date
    - *DOB* -> date
    - *list_price* -> float
    - *standart_cost* -> float
- получаем 4 таблицы
- данные будут приведены к 3НФ

# 5. Создаем итоговые таблицы

## 5.1. Создаем таблицу `transaction`

In [ ]:
-- создадим новую базу данных
create database homework_1;

-- создадим таблицу transaction
create table transaction (
    transaction_id integer not NULL,
	product_id integer,
	customer_id integer,
	transaction_date varchar(50),
	online_order boolean,
	order_status varchar(50)
);
-- создаем первичный ключ
alter table transaction add primary key (transaction_id);

Импортируем данные из `transaction.csv`:

<img src="imgs/image08.png" width="1200"/> 

<img src="imgs/image09.png" width="1200"/> 

In [ ]:
-- добавим колонку address_id
alter table transaction add address_id integer not null default 1;

## 5.2. Создаем таблицу `product`:

In [ ]:
-- создадим таблицу product
create table product (
    product_id integer not null,
    brand varchar(50),
    product_line varchar(50),
    product_class varchar(50),
    product_size varchar(50),
    list_price varchar(50),
    standard_cost varchar(50)
);

Импортируем данные из `transaction.csv`:

<img src="imgs/image10.png" width="1200"/> 

<img src="imgs/image11.png" width="1200"/> 

## 5.3. Создадим таблице `customer`:

In [ ]:
-- создадим таблицу customer
create table customer (
    customer_id integer,
  first_name varchar(50),
  last_name varchar(50),
  gender varchar(50),
  DOB varchar(50),
  job_title varchar(50),
  job_industry_category varchar(50),
  wealth_segment varchar(50),
  deceased_indicator varchar(50),
  owns_car varchar(50),
  property_valuation integer
);

Импортируем в нее необходимые данные из `customer.csv`:

<img src="imgs/image12.png" width="1200"/> 

<img src="imgs/image13.png" width="1200"/> 

In [ ]:
-- забыл указать, что customer_id не может принимать нулевое значение
alter table customer alter column customer_id set not null;

## 5.4. Создаем таблицу `address`:

In [ ]:
-- создадим таблицу address
create table address (
  address varchar(50)
  postcode integer,
  state varchar(50),
  country varchar(50)
);

Импортируем в нее необходимые данные из `customer.csv`:

<img src="imgs/image144.png" width="1200"/> 

<img src="imgs/image155.png" width="1200"/> 

In [ ]:
-- добавим колонку address_id
alter table address add address_id integer not null default 1;

# 6. Дополнительно

- В таблицах `product`, `customer`, `address` не выставлял первичные ключи, т.к. загрузка данных из исходных файлов приводит к дублированию атрибутов `_id` и первичный ключ не выставляется; либо если выставить его в пустой таблице, а уже затем грузить данные - возникнет ошибка (будут опять-таки дубликаты)
- Возможно, я неправильно понял задание и нужно было просто создать тестовые .csv с несколькими строчками уникальных даннных или использовать пару уникальных `insert into ... values ...` - тогда, естествено, все работает нормально и ключи выставляются